In [9]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-8B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

config.json: 0.00B [00:00, ?B/s]

You are using a model of type qwen3_vl to instantiate a model of type qwen2_5_vl. This is not supported for all configurations of models and can yield errors.
Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_interleaved'}


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/490 [00:00<?, ?it/s]

Qwen2_5_VLForConditionalGeneration LOAD REPORT from: Qwen/Qwen3-VL-8B-Instruct
Key                                                            | Status     | 
---------------------------------------------------------------+------------+-
model.language_model.layers.{0...35}.self_attn.q_norm.weight   | UNEXPECTED | 
model.visual.patch_embed.proj.bias                             | UNEXPECTED | 
model.visual.blocks.{0...26}.mlp.linear_fc2.bias               | UNEXPECTED | 
model.visual.blocks.{0...26}.mlp.linear_fc2.weight             | UNEXPECTED | 
model.language_model.layers.{0...35}.self_attn.k_norm.weight   | UNEXPECTED | 
model.visual.blocks.{0...26}.mlp.linear_fc1.weight             | UNEXPECTED | 
model.visual.blocks.{0...26}.norm2.bias                        | UNEXPECTED | 
model.visual.blocks.{0...26}.norm1.bias                        | UNEXPECTED | 
model.visual.merger.linear_fc1.bias                            | UNEXPECTED | 
model.visual.deepstack_merger_list.{0, 1, 2}.linear_

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [7]:
with open('scratch_web.html', 'r') as f:
    html = f.read()

print(html[:200])

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>User Profile</title>
    <style>
    * {
    ma


In [8]:
# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "scratch_ss.png",
            },
            {"type": "text", "text": "Give HTML and CSS for the reference UI."},
        ],
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": html},
        ],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "scratch_ss2.png",
            },
            {"type": "text", "text": "Rendered image from your code is given above. Update HTML + CSS based on the feedback from render to make it exactly like reference UI"},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=4096)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

NameError: name 'process_vision_info' is not defined

In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor, AutoConfig
import torch

model_name = "Qwen/Qwen3-VL-8B-Instruct"

# 1. Load config FIRST
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)

# 2. Patch the broken field
if not hasattr(config.text_config, "pad_token_id"):
    config.text_config.pad_token_id = 0   # Qwen uses 0 as pad by default

# (optional but safe)
if not hasattr(config, "pad_token_id"):
    config.pad_token_id = 0

# 3. Now load model with patched config
# model = Qwen3VLForConditionalGeneration.from_pretrained(
#     model_name,
#     config=config,
#     torch_dtype="auto",
#     device_map="auto",
#     trust_remote_code=True
# )


# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-8B-Instruct",
    dtype=torch.bfloat16,
    config=config,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-8B-Instruct")

Loading weights:   0%|          | 0/750 [00:00<?, ?it/s]

In [2]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "scratch_ss.png",
            },
            {"type": "text", "text": "Give HTML and CSS for the reference UI."},
        ],
    },
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=4096)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>User Profile</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.6;
            color: #333;
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f9f9f9;
        }
        
        h1, h2, h3 {
            color: #2c3e50;
        }
        
        .section {
            margin-bottom: 30px;
        }
        
        .section h2 {
            margin-top: 0;
            padding-bottom: 10px;
            border-bottom: 1px solid #e0e0e0;
        }
        
        .badge-container {
            display: flex;
            flex-wrap: wrap;
            gap: 20px;
            margin-top: 20px;
        }
        
        .badge-card {
            flex: 1 1 300px;
            background: #f